# Data Download

In [1]:
pwd

'/storage/yangjianLab/chenwenhao/projects/202312_GPS/src/GPS/docs/source'

In [2]:
%env PATH=/storage/yangjianLab/chenwenhao/Software/micromamba/envs/torch_v2/bin:/storage/yangjianLab/chenwenhao/Software/google-cloud-sdk/bin:/home/yangjianLab/chenwenhao/bin:/storage/yangjianLab/chenwenhao/bin:/opt/slurm/sbin:/opt/slurm/bin:/soft/singularity-3.7.1/bin:/storage/yangjianLab/chenwenhao/Software/micromamba/condabin:/storage/yangjianLab/chenwenhao/Software/google-cloud-sdk/bin:/home/yangjianLab/chenwenhao/bin:/storage/yangjianLab/chenwenhao/bin:/opt/slurm/sbin:/opt/slurm/bin:/home/yangjianLab/chenwenhao/.cargo/bin:/home/yangjianLab/chenwenhao/bin/bin:/home/yangjianLab/chenwenhao/.local/bin:/home/yangjianLab/chenwenhao/bin:/opt/slurm/sbin:/opt/slurm/bin:/soft/modules/modules-4.7.0/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/home/yangjianLab/chenwenhao/bin

env: PATH=/storage/yangjianLab/chenwenhao/Software/micromamba/envs/torch_v2/bin:/storage/yangjianLab/chenwenhao/Software/google-cloud-sdk/bin:/home/yangjianLab/chenwenhao/bin:/storage/yangjianLab/chenwenhao/bin:/opt/slurm/sbin:/opt/slurm/bin:/soft/singularity-3.7.1/bin:/storage/yangjianLab/chenwenhao/Software/micromamba/condabin:/storage/yangjianLab/chenwenhao/Software/google-cloud-sdk/bin:/home/yangjianLab/chenwenhao/bin:/storage/yangjianLab/chenwenhao/bin:/opt/slurm/sbin:/opt/slurm/bin:/home/yangjianLab/chenwenhao/.cargo/bin:/home/yangjianLab/chenwenhao/bin/bin:/home/yangjianLab/chenwenhao/.local/bin:/home/yangjianLab/chenwenhao/bin:/opt/slurm/sbin:/opt/slurm/bin:/soft/modules/modules-4.7.0/bin:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/home/yangjianLab/chenwenhao/bin


In [4]:
%%bash
GPS -h

usage: GPS [-h] [--version]
           {run_find_latent_representations,run_latent_to_gene,run_generate_ldscore,run_spatial_ldsc,run_cauchy_combination,run_visualize,run_all_mode}
           ...

 GPS: Genetics-informed pathogenic spatial mapping

options:
  -h, --help            show this help message and exit
  --version, -v         show program's version number and exit

Available subcommands:
  {run_find_latent_representations,run_latent_to_gene,run_generate_ldscore,run_spatial_ldsc,run_cauchy_combination,run_visualize,run_all_mode}
                        Subcommands
    run_find_latent_representations
                        Run Find_latent_representations 
                        Find the latent representations of each spot by running GNN-VAE
    run_latent_to_gene  Run Latent_to_gene 
                        Find gene marker gene scores for each spot by using latent representations from nearby spots
    run_generate_ldscore
                        Run Generate_ldscore 
        

In [5]:
!which python

/storage/yangjianLab/chenwenhao/Software/micromamba/envs/torch_v2/bin/python


In [6]:
!GPS -v

GPS version 1.51


In [11]:
%%bash

# Constants and configuration
WORKDIR='./example/Human_Cortex' # This should be the directory where the GPS output will be saved
SAMPLE_NAME="Cortex_151507" # This should be the name of the sample

# Input data
HDF5_PATH="/storage/yangjianLab/songliyang/SpatialData/Data/Brain/Human/Nature_Neuroscience_2021/processed/h5ad/Cortex_151507.h5ad"
ANNOTATION="layer_guess" # This should be the cell type annotation of each spot
DATA_TYPE='count' # This should be the type of ST data, either 'count' or 'log1p'

# Running Dependencies and Resources
GTFFILE="/storage/yangjianLab/songliyang/ReferenceGenome/GRCh37/gencode.v39lift37.annotation.gtf"
BFILE_ROOT="/storage/yangjianLab/sharedata/LDSC_resource/1000G_EUR_Phase3_plink/1000G.EUR.QC"
KEEP_SNP_ROOT="/storage/yangjianLab/sharedata/LDSC_resource/hapmap3_snps/hm"
BASELINE_ANNOTATION_DIR='/storage/yangjianLab/chenwenhao/projects/202312_GPS/data/resource/ldsc/baseline_v1.2/remove_base'
W_FILE="/storage/yangjianLab/sharedata/LDSC_resource/LDSC_SEG_ldscores/weights_hm3_no_hla/weights."

# GWAS Summary Statistics
SUMSTATS_CONFIG_FILE='/storage/yangjianLab/chenwenhao/projects/202312_GPS/src/GPS/example/sumstats_config_sub.yaml'

In [16]:
%%bash??

In [17]:
%%bash

# find_latent_representations
HDF5_OUTPUT="$WORKDIR/$SAMPLE_NAME/find_latent_representations/${SAMPLE_NAME}_add_latent.h5ad"
GPS run_find_latent_representations \
    --input_hdf5_path $HDF5_PATH \
    --sample_name $SAMPLE_NAME \
    --output_hdf5_path $HDF5_OUTPUT \
    --annotation $ANNOTATION \
    --type $DATA_TYPE \
    [OTHER PARAMETERS HERE]

# latent_to_gene
HDF5_WITH_LATENT_PATH="$WORKDIR/$SAMPLE_NAME/find_latent_representations/${SAMPLE_NAME}_add_latent.h5ad"
FEATHER_PATH="$WORKDIR/$SAMPLE_NAME/latent_to_gene/${SAMPLE_NAME}_gene_marker_score.feather"
GPS run_latent_to_gene \
    --input_hdf5_with_latent_path $HDF5_WITH_LATENT_PATH \
    --sample_name $SAMPLE_NAME \
    --output_feather_path $FEATHER_PATH \
    [OTHER PARAMETERS HERE]

# generate_ldscore
for CHROM in {1..22}; do
    MKSCORE_FEATHER_FILE="$WORKDIR/$SAMPLE_NAME/latent_to_gene/${SAMPLE_NAME}_gene_marker_score.feather"
    DONE_PATH="$WORKDIR/$SAMPLE_NAME/generate_ldscore/${SAMPLE_NAME}_generate_ldscore_chr${CHROM}.done"
    GPS run_generate_ldscore \
        --sample_name $SAMPLE_NAME \
        --chrom $CHROM \
        --ldscore_save_dir "$WORKDIR/$SAMPLE_NAME/generate_ldscore" \
        --mkscore_feather_file $MKSCORE_FEATHER_FILE \
        --bfile_root $BFILE_ROOT \
        --keep_snp_root $KEEP_SNP_ROOT \
        --gtf_annotation_file $GTFFILE \
        [OTHER PARAMETERS HERE]
    touch $DONE_PATH
done

# spatial_ldsc
LDSCORE_INPUT_DIR="$WORKDIR/$SAMPLE_NAME/generate_ldscore"
LDSC_SAVE_DIR="$WORKDIR/$SAMPLE_NAME/spatial_ldsc"
GPS run_spatial_ldsc --w_file $W_FILE --sample_name $SAMPLE_NAME [OTHER PARAMETERS HERE]
touch "$WORKDIR/$SAMPLE_NAME/spatial_ldsc/${SAMPLE_NAME}.spatial_ldsc.done"

# cauchy_combination
for TRAIT_NAME in [LIST TRAIT NAMES HERE]; do
    LDSC_DIR="$WORKDIR/$SAMPLE_NAME/spatial_ldsc"
    CAUCHY_SAVE_DIR="$WORKDIR/$SAMPLE_NAME/cauchy_combination"
    GPS run_cauchy_combination --input_hdf5_path $HDF5_PATH --input_ldsc_dir $LDSC_DIR --sample_name $SAMPLE_NAME --output_cauchy_dir $CAUCHY_SAVE_DIR --trait_name $TRAIT_NAME --annotation $ANNOTATION
done


usage: GPS run_find_latent_representations [-h] --input_hdf5_path
                                           INPUT_HDF5_PATH --output_hdf5_path
                                           OUTPUT_HDF5_PATH --sample_name
                                           SAMPLE_NAME
                                           [--annotation ANNOTATION]
                                           [--type TYPE] [--epochs EPOCHS]
                                           [--feat_hidden1 FEAT_HIDDEN1]
                                           [--feat_hidden2 FEAT_HIDDEN2]
                                           [--feat_cell FEAT_CELL]
                                           [--gcn_hidden1 GCN_HIDDEN1]
                                           [--gcn_hidden2 GCN_HIDDEN2]
                                           [--p_drop P_DROP] [--gcn_lr GCN_LR]
                                           [--gcn_decay GCN_DECAY]
                                           [--n_neighbors N_NEIGHBORS]
           

CalledProcessError: Command 'b'\n# find_latent_representations\nHDF5_OUTPUT="$WORKDIR/$SAMPLE_NAME/find_latent_representations/${SAMPLE_NAME}_add_latent.h5ad"\nGPS run_find_latent_representations \\\n    --input_hdf5_path $HDF5_PATH \\\n    --sample_name $SAMPLE_NAME \\\n    --output_hdf5_path $HDF5_OUTPUT \\\n    --annotation $ANNOTATION \\\n    --type $DATA_TYPE \\\n    [OTHER PARAMETERS HERE]\n\n# latent_to_gene\nHDF5_WITH_LATENT_PATH="$WORKDIR/$SAMPLE_NAME/find_latent_representations/${SAMPLE_NAME}_add_latent.h5ad"\nFEATHER_PATH="$WORKDIR/$SAMPLE_NAME/latent_to_gene/${SAMPLE_NAME}_gene_marker_score.feather"\nGPS run_latent_to_gene \\\n    --input_hdf5_with_latent_path $HDF5_WITH_LATENT_PATH \\\n    --sample_name $SAMPLE_NAME \\\n    --output_feather_path $FEATHER_PATH \\\n    [OTHER PARAMETERS HERE]\n\n# generate_ldscore\nfor CHROM in {1..22}; do\n    MKSCORE_FEATHER_FILE="$WORKDIR/$SAMPLE_NAME/latent_to_gene/${SAMPLE_NAME}_gene_marker_score.feather"\n    DONE_PATH="$WORKDIR/$SAMPL